In [6]:
import pycutest
import numpy as np
import time
import pandas as pd

testes = [
    "ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
    "FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
    "HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
    "LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
    "QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
    "TOINTGOR"
]

In [7]:
class parameters:
    def __init__(self,function: str):
        self.function = pycutest.import_problem(function)                #deixar so os metodos publicos depois
        self.xk = self.function.x0                                      
        self.old_xk = None
                                               #Algoritmo de descida
        self.grad = self.function.grad(self.xk)                          #sempre teremos uma funcao gradiente para computar
        self.grad_calls = 1                                              #por isso inicializamos a chamada de gradiente como 1
        self.val_calls = 0
        self.old_grad = None                                       #algoritmo de busca

        #--------------------------------Classe fila para Armijo Modificada--------------------------
        
    def get_new_point(self,x):
        self.old_xk = self.xk
        self.xk = x
        self.old_grad = self.grad
        self.grad = self.function.grad(self.xk)
        self.grad_calls += 1
        
    def gradient(self):
        return self.grad
    
    def objective(self, ponto):
        self.val_calls += 1
        return self.function.obj(ponto)        

In [8]:
def Armijo(function: parameters, passo: float, dk, a, b) ->bool:
    aux = function.objective(function.xk + passo*dk)                  #utilizar armijo so para verificar                        
    if (aux <= passo*a + b):                                          #se o passo e bom ou nao
        return True
    return False

#--------------------------------------------------------------------------------------------------
#Busca usada
def step_parameter(parameters: parameters, direction) -> float:

    M = parameters.objective(parameters.xk)
    passo = 1.0
    aux = 1e-4*np.dot(parameters.grad,direction)                            #parametro eta = 1e-4 referencia relatorio
    for j in range(100):                                                    #limite j = 100 dificilmente chega nele, um numero exagerado de grande
        if Armijo(parameters, passo,direction,aux,M):                       #explicação detalhada esta no relatorio
            return passo
        else:
            passo = passo*0.5                                               #parametro beta que está no relatorio a referencia
    return passo                                                            #mas qualquer numero entre 0,5 a 0,8 converge bem

def minimize(function: parameters, tolerance = 1e-5):            
    maximum_iterations = int(1e5)
    start = time.perf_counter()                                         
    for iteration in range(maximum_iterations):
        if np.linalg.norm(function.gradient(),ord=np.inf) < tolerance:
            end = time.perf_counter()
            tempo = end - start
            return {
                "val_calls": function.val_calls,
                "grad_calls": function.grad_calls,
                "tempo": tempo
            }
        else:
            dk = -function.gradient()
            step = step_parameter(function,dk)
            function.get_new_point(function.xk + step*dk)
    # Retorna negativo para indicar falha, se não convergir em máximo de iterações
    return {
        "val_calls": -function.val_calls,
        "grad_calls": -function.grad_calls,
        "tempo": 0.0
    }

In [ ]:
Gradiente = {i: minimize(parameters(i)) for i in testes}

In [74]:
df = pd.DataFrame(Gradiente).T
df

,val_calls,grad_calls,tempo
ARGLINA,3.0,2.0,0.000957
ARGLINB,-688648.0,-10001.0,0.000000
BA-L1SPLS,2119.0,85.0,0.061687
BIGGS6,-42586.0,-10001.0,0.000000
BROWNAL,97236.0,5720.0,15.671750
COATING,-138911.0,-10001.0,0.000000
FLETCHCR,-117700.0,-10001.0,0.000000
GAUSS2LS,-332116.0,-10001.0,0.000000
GENROSE,-117697.0,-10001.0,0.000000
HAHN1LS,-438600.0,-10001.0,0.000000


In [75]:
df.to_csv(r"grad_tabela.csv",index=False)

In [5]:
Gradiente = {i: minimize(parameters(i)) for i in testes}
df2 = pd.DataFrame(Gradiente).T
df2.to_csv(r"grad_tabela_melhorada.csv",index=False)
df2

,val_calls,grad_calls,tempo
ARGLINA,3.0,2.0,0.021708
ARGLINB,-6898648.0,-100001.0,0.000000
BA-L1SPLS,2321.0,93.0,0.061657
BIGGS6,-433366.0,-100001.0,0.000000
BROWNAL,146043.0,8591.0,25.390956
COATING,-1392362.0,-100001.0,0.000000
FLETCHCR,209140.0,17769.0,22.875224
GAUSS2LS,-3318694.0,-100001.0,0.000000
GENROSE,268068.0,22775.0,19.746842
HAHN1LS,-4731468.0,-100001.0,0.000000
